In [1]:
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas_profiling
from pandas_profiling import ProfileReport
%matplotlib inline

from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from googletrans import Translator
from sklearn.impute import SimpleImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.impute import KNNImputer

from imblearn.over_sampling import SMOTE

from fancyimpute import KNN, IterativeImputer

from xgboost import XGBClassifier
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Missing Data Review

Given that the dataset has now been cleaned, we can move onto reviewing missing data. 

In [2]:
# Load cleaned data
df = pd.read_pickle('cleaned_df.pkl')

# Review missing across rows
print(df.count(axis=1).value_counts())

12    37855
13    22357
10    16636
11    13939
9      4652
7      2220
8      1852
14      849
6       456
4       177
5       122
dtype: int64


First, we can drop rows with a certain level missing. Given that there are 14 columns, removing those with less than 10 completed would still keep a large portion of the data but give us a clearer picture.

In [4]:
# Given 14 columns, drop rows with 10 or less features
df.dropna(thresh=10,axis=0, inplace=True)

# Review missing data - columms
print(df.isna().sum()/len(df))

other_cat1      0.019239
other_cat2      0.029672
other_cat3      0.002739
other_cat4      0.002739
ordinal1        0.231699
type            0.372223
location3       0.000000
location4       0.000000
location5       0.011633
ordinal2        0.558623
other_cat5      0.270014
sector2         0.754114
label           0.000000
adj_m_signup    0.000000
dtype: float64


We can review those missing variables more closely.

In [5]:
### Review Conversion Rate Between Missing Vars
missing_cols = df.columns[df.isnull().any()]

# Find Percentages of Most Frequent
def freq_perc(col):
    x = df[col].dropna()
    max_val = x.value_counts().idxmax()
    max_perc = x.value_counts().max() / len(x)
    missing_perc = 1-(len(x.dropna())/len(df))
    print('{}\nMost Frequent: {}, Percentage: {:.4f}'.format(col, max_val, max_perc))
    print('Missing {:.4f}\n'.format(missing_perc))

[freq_perc(x) for x in missing_cols]

other_cat1
Most Frequent: Signup, Percentage: 0.9808
Missing 0.0192

other_cat2
Most Frequent: Signup, Percentage: 0.9430
Missing 0.0297

other_cat3
Most Frequent: Google, Percentage: 0.5067
Missing 0.0027

other_cat4
Most Frequent: Home, Percentage: 0.3274
Missing 0.0027

ordinal1
Most Frequent: 1.0, Percentage: 0.9117
Missing 0.2317

type
Most Frequent: isp, Percentage: 0.9681
Missing 0.3722

location5
Most Frequent: 2.0, Percentage: 0.1854
Missing 0.0116

ordinal2
Most Frequent: 5.0, Percentage: 0.3839
Missing 0.5586

other_cat5
Most Frequent: developer, Percentage: 0.7529
Missing 0.2700

sector2
Most Frequent: Information Technology, Percentage: 0.5453
Missing 0.7541



[None, None, None, None, None, None, None, None, None, None]

For those categorical variables that are missing, the most frequent inputs either have a very large proportion, or the missingness of the variable itself is very low. However, sector is both highly missing and it's most frequent input only accounts for about half of the input set. We can impute the most frequent for all categorical variables outisde of category sector for the time being. 

For those continuous variables that are missing, ordinal1 is highly skewed so most frequent imputation would make sense. For ordinal2, we can impute the median value. 

In [12]:
##### Use most frequent for section of columns
freq_cols = [x for x in missing_cols if x not in ['ordinal2','sector2']]

for col in freq_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

##### Use median for employee size (continuous var); 
df['ordinal2'].fillna(df.ordinal2.dropna().median(), inplace=True)

# Review missing
df.columns[df.isnull().any()]

Index(['sector2'], dtype='object')

For imputing sectors, we can use KNN imputation to find those combination of other columns that are most similar and impute the value found therein for category. To do this, I will use an encoding process that allows categorical features to be encoded for the imputation process, and then inverse transformed to keep the original string values for further modeling. 

In [13]:
class MultiColumnLabelEncoder:

    def __init__(self, columns=None):
        self.columns = columns 


    def fit(self, X, y=None):
        self.encoders = {}
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            self.encoders[col] = LabelEncoder().fit(X[col])
        return self


    def transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].transform(X[col])
        return output


    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)


    def inverse_transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].inverse_transform(X[col])
        return output

In [15]:
# Specify categorical columns
cat_cols = ['type', 'other_cat1', 'other_cat2', 'other_cat3', 'other_cat4', 'other_cat5']

other = 'sector2'

# Force all to string
for col in cat_cols:
    df[col] = df[col].astype(str)
    
# Instantiate Label Encoder
multi = MultiColumnLabelEncoder(columns=cat_cols)

# Create new df
df_encode = df.copy()

# Encode
df_encode = multi.fit_transform(df_encode)

# Encode just sectors
le = LabelEncoder()
df_encode.sector2 = df_encode.sector2.astype(str)
df_encode.sector2 = le.fit_transform(df_encode.sector2)

# Re-establish NA
df_encode = df_encode.mask(df.isna())

# Imputation on sectors
imputer = KNNImputer(n_neighbors=2)
df_impute = pd.DataFrame(imputer.fit_transform(df_encode), 
                        columns = df_encode.columns)

# Establish levels for sector
df_impute.sector2 = [int(x) for x in df_impute.sector2]

# Save pickle
df_impute.to_pickle('df_impute.pkl')

####### Inverse Transform - get labeled data
# Recode sector
df_impute.sector2 = le.inverse_transform(df_impute.sector2)

# Inverse transform cat cols
df_impute[cat_cols] = multi.inverse_transform(df_impute[cat_cols].astype(int))

# Save pickle
df_impute.to_pickle('df_impute_label.pkl')

### SMOTE

Currently, the dataset is quite imbalanced in regard to conversion. To alleviate this, we can use SMOTE oversampling techniques to create synthetic converted data. In using this data, we will also need to set aside a portion of the original set to validate, as SMOTE can often overfit. 

In [18]:
# Use SMOTE Oversampling
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTEENN

# Load Imputed
df_impute = pd.read_pickle('df_impute.pkl')

# Specify X, y
X = df_impute.drop(columns = 'label')
y = df_impute.label

# Set aside non oversampled for testing
X_train, X_validate, y_train, y_validate = train_test_split(X, y,
                                                stratify=y, 
                                                test_size=0.33)

# Create validation set for later
df_validate = X_validate
df_validate['label']=y_validate
df_validate.to_pickle('df_validate.pkl')

# Reestablish dataframe to oversample
X = X_train
y = y_train

# Specify overall categorical columns
full_cat_cols = [x for x in df_impute.columns if x not in ['ordinal1', 'ordinal2', 'label']]

# Get categorical column indices
cat_col_idx = [X.columns.get_loc(c) for c in full_cat_cols if c in X]

# Create over and under
sme = SMOTENC(categorical_features=cat_col_idx, sampling_strategy=0.33)
X_res, y_res = sme.fit_resample(X, y)

# Check counts
print(y.value_counts())
print(y_res.value_counts())

0.0    60688
1.0      708
Name: label, dtype: int64
0.0    60688
1.0    20027
Name: label, dtype: int64


In [19]:
# Create oversampled df
df_smote = X_res.copy()
df_smote['label'] = y_res

# Save to pickle
df_smote.to_pickle('df_smote.pkl')